# Metodo Frank & Wolfe
Questo file risolve il problema dell'equilibrio su reti usato con la base del libro "Data Networks,"
(2nd edition) Prentice Hall, 1992, ISBN 0132009161. Per la risoluzione di questo problema si usa
l'algoritmo di Frank & Wolfe per la risoluzione del problema di ottimizzazione che ne deriva dal
calcolo di equilibrio su reti. Non approfondisco le ipotesi che portano a questa formulazione,
sono molto semplici.

In [318]:
import sympy as sym
from sympy import init_printing
import warnings

warnings.filterwarnings("ignore", category=UserWarning)
from IPython.display import display, Math

x1, x2, x3 = sym.symbols('x1, x2, x3')
x = sym.Matrix([[x1], [x2], [x3]])
Q = sym.Matrix([[1, 0, 0], [0, 1, 0], [0, 0, 0.1]])
c = sym.Matrix([[0], [0], [0.55]])

Adesso posso visualizzare la funzione obiettivo come espressione bellina


In [319]:
init_printing()

obj = 0.5 * (x.T * Q * x) + c.T * x
obj

⎡      2         2          2          ⎤
⎣0.5⋅x₁  + 0.5⋅x₂  + 0.05⋅x₃  + 0.55⋅x₃⎦

Adesso visualizzo anche l'obiettivo al punto iniziale:

In [320]:
cost = obj.subs({x1: 0.4, x2: 0.3, x3: 0.3})
display(Math(r'D(x_0): {:.4f}'.format(cost[0])))

<IPython.core.display.Math object>

Adesso posso visualizzare la funzione con lo step size e l'aggiornamento

In [321]:
x1_bar, x2_bar, x3_bar = sym.symbols('\overline{x_1}, \overline{x_2}, \overline{x_3}')
x_bar = sym.Matrix([[x1_bar], [x2_bar], [x3_bar]])
d_k = x_bar - x
display(Math(r'd_k: '))
d_k

<IPython.core.display.Math object>

⎡\overline{x_1} - x₁⎤
⎢                   ⎥
⎢\overline{x_2} - x₂⎥
⎢                   ⎥
⎣\overline{x_3} - x₃⎦

Adesso visualizzo il punto iniziale e mi calcolo il nuovo punto

In [322]:
a = sym.symbols('\\alpha')
x_k = x + a * d_k

display(Math(r'x_{k+1}: '))
x_k

<IPython.core.display.Math object>

⎡\alpha⋅(\overline{x_1} - x₁) + x₁⎤
⎢                                 ⎥
⎢\alpha⋅(\overline{x_2} - x₂) + x₂⎥
⎢                                 ⎥
⎣\alpha⋅(\overline{x_3} - x₃) + x₃⎦

Adesso mi calcolo il gradiente

In [323]:
grad = x.T * Q + c.T
display(Math(r'\nabla: '))
grad.T

<IPython.core.display.Math object>

⎡     x₁      ⎤
⎢             ⎥
⎢     x₂      ⎥
⎢             ⎥
⎣0.1⋅x₃ + 0.55⎦

Adesso mi calcolo ${\alpha}$ come forma chiusa e nella cella successiva sostituisco i valori:

In [324]:
alpha = -(grad * d_k) / (d_k.T * Q * d_k)[0]
alpha

⎡     -x₁⋅(\overline{x_1} - x₁) - x₂⋅(\overline{x_2} - x₂) - (\overline{x_3} -
⎢─────────────────────────────────────────────────────────────────────────────
⎢                     2                        2                              
⎣(\overline{x_1} - x₁)  + (\overline{x_2} - x₂)  + (0.1⋅\overline{x_3} - 0.1⋅x

 x₃)⋅(0.1⋅x₃ + 0.55)    ⎤
────────────────────────⎥
                        ⎥
₃)⋅(\overline{x_3} - x₃)⎦

Adesso sostituisco i valori e verifico:

    1. I valori tornano con Gallagher
    2. La direzione scelta con Q
    3. Calcolo del nuovo punto con il quale calcolare la direzione di discesa
\begin{align} \overline{x}_{k+1} = argmin {<\nabla f(x_k), x>} \quad \forall	 x \in C \end{align}

Punto iniziale scelto per far partire l'algoritmo, uso il seguente:

In [325]:
result_x = x.subs({x1: 0.4, x2: 0.3, x3: 0.3})
result_x

⎡0.4⎤
⎢   ⎥
⎢0.3⎥
⎢   ⎥
⎣0.3⎦

Uso questa soluzione iniziale:

In [326]:
result_x_bar = x_bar.subs({x1_bar: 1, x2_bar: 0, x3_bar: 0})
result_x_bar

⎡1⎤
⎢ ⎥
⎢0⎥
⎢ ⎥
⎣0⎦

Mi calcolo la nuova direzione

In [327]:
result_d_k = d_k.subs({x1: 0.4, x2: 0.3, x3: 0.3, x1_bar: 1, x2_bar: 0, x3_bar: 0})
result_d_k

⎡0.6 ⎤
⎢    ⎥
⎢-0.3⎥
⎢    ⎥
⎣-0.3⎦

Adesso mi calcolo lo step size con la line minimization

In [328]:
step_result = alpha.subs({x1: 0.4, x2: 0.3, x3: 0.3, x1_bar: 1, x2_bar: 0, x3_bar: 0})
step_result

[0.0522875816993464]

Il nuovo punto è

In [329]:
x_k.subs({x1: 0.4, x2: 0.3, x3: 0.3, a: step_result, x1_bar: 1, x2_bar: 0, x3_bar: 0})

⎡0.4 + [0.0313725490196079] ⎤
⎢                           ⎥
⎢0.3 + [-0.0156862745098039]⎥
⎢                           ⎥
⎣0.3 + [-0.0156862745098039]⎦